# Model Project: Implementation of the Binomial Option Pricing model proposed by Cox, Ross and Rubinstein in 1979

The binomial option pricing model is widely used in finance to value options on stocks as well as other assets. The model was invented in 1979 by Cox, Ross and Rubinstein and an in depth walk through of the model can be found in the following article on Wikipedia: https://en.wikipedia.org/wiki/Binomial_options_pricing_model

## Importing packages

In [1]:
import numpy as np
import math as math

## Defining the binomial asset pricing model

First the binomial pricing model is defined. The model takes in four inputs; the amount of periods in the model (n), the time duration of each period (t), the current normalizaed price of the asset (S) as well as the variance in the price (v). 

By each period of the model the price of the asset can either go up (u) or down (d). The outcome of each of the two states are presented below:

\\[ u = e^{v \cdot \sqrt{dt}}\\]
\\[ d = e^{-v \cdot \sqrt{dt}} = \frac{1}{u} \\]

Further the price of the asset is defined by the following equation, which is a function of u and d:
\\[ S_n = S_{0} \cdot u^{N_u - N_d}     \\]
With the amount of up and down ticks: \\[ N_u \quad and \quad N_d \\] 


In [2]:
def Binomial_Pricing_Model(n,t,S,v):
    dt = t/n
    u = math.exp(v*math.sqrt(dt))
    d = 1/u
    asset_prices = np.zeros((n+1, n+1))
    for j in range(n+1):
        for i in range(j+1):
            asset_prices[i,j] = S*math.pow(d,i) * math.pow(u,j-i)
    return asset_prices

We then feed in the model inputs, setting the amount of periods equal to 10, duration of each period to be 200 workdays pr. year, the asset price to 100 and the variance to 0,5. The model is then called and the result is printed below:

In [3]:
n = 10
t = 200/365
S = 100
v = .5

x = Binomial_Pricing_Model(n,t,S,v)

print('For n = 10 the price evolves as:\n',np.matrix(x.astype(int)))

For n = 10 the price evolves as:
 [[100 112 126 142 159 179 201 226 255 286 322]
 [  0  88 100 112 126 142 159 179 201 226 255]
 [  0   0  79  88 100 112 126 142 159 179 201]
 [  0   0   0  70  79  88 100 112 126 142 159]
 [  0   0   0   0  62  70  79  88 100 112 126]
 [  0   0   0   0   0  55  62  70  79  88 100]
 [  0   0   0   0   0   0  49  55  62  70  79]
 [  0   0   0   0   0   0   0  44  49  55  62]
 [  0   0   0   0   0   0   0   0  39  44  49]
 [  0   0   0   0   0   0   0   0   0  34  39]
 [  0   0   0   0   0   0   0   0   0   0  31]]


We notice that over 10 periods the maximum price the it stock evolves to is 322, as well as the lowest possible outcome is 31. These are however extreme observations and it is more likely that the final stock price will end up somewhere in the middle. 

## Defining the binomial option pricing model
With the asset price function from above we can now state the option pricing model as per Cox, Ross and Rubinstein. The model uses backwards induction to iteratively price the option one period back in time. 
Calculating the probabilities of either the up or down state the model uses the risk neautral probability measure q which is extracted from a Brownian Motion:
\\[ q = \frac{e^{r \cdot dt} - d}{u-d}    \\]


### Pricing of the call option

First we define the function valuing a call option on the asset. Aside from taking in the q-probabilities and the inputs from above, the pricing model also utilizes the strike price k and the risk-free interest rate. 

The function pricing the call option (CallValue), which is presented below constantly, initially fills out the entire matrix, which is n by n. Afterwards, the function Cm at each note solves the following maximisation problem:
\\[Max[(S_n-k),0]     \\]


In [4]:
def CallValue(n, S, k, r, v, t):
    dt = t/n                    
    u = math.exp(v*math.sqrt(dt)) 
    d = 1/u                     
    q = (math.exp(r*dt)-d)/(u-d)   
    Pm = np.zeros((n+1, n+1))   
    Cm = np.zeros((n+1, n+1))
    tmp = np.zeros((2,n+1))
    for j in range(n+1):
        tmp[0,j] = S*math.pow(d,j)
        tmp[1,j] = S*math.pow(u,j)
    tot = np.unique(tmp)
    c = n

    for i in range(c+1):
        for j in range(c+1):
            Pm[i,j-c-1] = tot[(n-i)+j]
        c=c-1
    for j in range(n+1, 0, -1):
        for i in range(j):
                if (j == n + 1):
                    Cm[i,j-1] = max(Pm[i,j-1]-k, 0)     
                else:
                    Cm[i,j-1] = math.exp(-.05*dt) * (q*Cm[i,j] + (1-q)*Cm[i+1,j])  
    return [Pm,Cm] 

We define the strike price (k) as well as the risk-free interest rate (r). These can be set as you please. The project is however intended to be run with a strike price of 102 and a risk free interest rate of 5%, why the comments below only apply to these inputs.

In [5]:
k = 102
r = .05

We then present the price of the call option, with a strike of 102, as each node in the binomial tree.

In [8]:
Pm,CmC = CallValue(n,S,k,r,v,t)
print('Call Option:\n',np.matrix(CmC.astype(int)))


Call Option:
 [[ 14  21  31  44  60  78 100 125 153 185 220]
 [  0   8  13  19  29  42  58  78 100 125 153]
 [  0   0   3   6  11  17  27  41  58  77  99]
 [  0   0   0   1   2   4   8  15  25  40  57]
 [  0   0   0   0   0   0   1   2   5  11  24]
 [  0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0]]


The results show that at the initial point the call option has a price of 14. From here on the price evolves according to the evolution of the underlying asset - as it moves up the price of the option increases, as it declines so does the price of the option. Note that the call option will only be utilized as the value of the stock is over 102. This due to the fact that there is no point in using an option to buy a stock at 102, when its underlying price is only 95, for instance. 

### Pricing of the corresponding put option

Again, we define the function valuing a the option on the asset, this time with a put rather than a call. 

The function pricing the put option (PutValue), which is presented below constantly, initially fills out the entire matrix, which is n by n. Afterwards, the function Cm at each note solves the following maximisation problem:
\\[Max[(k-S_n),0]     \\]


In [36]:
def PutValue(n, S, K, r, v, t):
    dt = t/n                    
    u = math.exp(v*math.sqrt(dt)) 
    d = 1/u                     
    p = (math.exp(r*dt)-d)/(u-d)   
    Pm = np.zeros((n+1, n+1))   
    Cm = np.zeros((n+1, n+1))
    tmp = np.zeros((2,n+1))
    for j in range(n+1):
        tmp[0,j] = S*math.pow(d,j)
        tmp[1,j] = S*math.pow(u,j)
    tot = np.unique(tmp)
    c = n
    
    for i in range(c+1):
        for j in range(c+1):
            Pm[i,j-c-1] = tot[(n-i)+j]
        c=c-1
    for j in range(n+1, 0, -1):
        for i in range(j):
                if(j == n+1):
                    Cm[i,j-1] = max(K-Pm[i,j-1], 0)     
                else:
                    Cm[i,j-1] = math.exp(-.05*dt) * (p*Cm[i,j] + (1-p)*Cm[i+1,j]) 
            
    return [Pm,Cm] 

The model takes in the same inputs for the strike price as well as risk free interest rate presented earlier. 

Again the results are presented in a matrix form as shown below.

In [37]:
Pm,CmP= PutValue(n, S, k, r, v, t)
print('Put Option:\n',np.matrix(CmP.astype(int)))

Put Option:
 [[14  9  5  2  0  0  0  0  0  0  0]
 [ 0 18 12  7  3  1  0  0  0  0  0]
 [ 0  0 24 17 11  5  2  0  0  0  0]
 [ 0  0  0 31 23 16  9  3  0  0  0]
 [ 0  0  0  0 38 30 23 14  7  1  0]
 [ 0  0  0  0  0 44 38 30 22 12  2]
 [ 0  0  0  0  0  0 51 45 38 31 22]
 [ 0  0  0  0  0  0  0 57 51 46 39]
 [ 0  0  0  0  0  0  0  0 62 57 52]
 [ 0  0  0  0  0  0  0  0  0 66 62]
 [ 0  0  0  0  0  0  0  0  0  0 70]]


Again we note that the initial price of the put option, with a strike of 102, is 14. This is due to the fact that the chance of going up and down are equal as well as the variance being equal to 0,5. From here on the put evovles and takes in higher and higher numbers the lower the pricing of the underlying asset becomes. In the case where the price of the asset falls at every opportunity, and in the end only is 31, the put option is worth 70. The reason that this numbers is not equal to 71 is due to round in the price if the underlying asset. 